<a href="https://colab.research.google.com/github/rajanpawar90/TensorFlow-in-Practice-Deeplearning.ai-/blob/master/SummaryCode/Course2_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
 #Install the latest version of tensorflow
 !pip install --upgrade tensorflow

In [0]:
#rock-paper-scissors data extracrt images from base directory zip files
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip \
    -O /tmp/rps.zip
  
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip \
    -O /tmp/rps-test-set.zip


 
  


#Data Preprocessing
Let's set up data generators that will read pictures in our source folders, convert them to float32 tensors, and feed them (with their labels) to our network. We'll have one generator for the training images and one for the validation images. Our generators will yield batches of 20 images of size 150x150 and their labels (binary).

As you may already know, data that goes into neural networks should usually be normalized in some way to make it more amenable to processing by the network. (It is uncommon to feed raw pixels into a convnet.) In our case, we will preprocess our images by normalizing the pixel values to be in the [0, 1] range (originally all values are in the [0, 255] range).

In Keras this can be done via the keras.preprocessing.image.ImageDataGenerator class using the rescale parameter. This ImageDataGenerator class allows you to instantiate generators of augmented image batches (and their labels) via .flow(data, labels) or .flow_from_directory(directory). These generators can then be used with the Keras model methods that accept data generators as inputs: fit_generator, evaluate_generator, and predict_generator.

In [0]:
######################################################################################################################################
#Function for generating ImageDataGenerator object for all datasets in this program. This datatype is necessay for fitting the model 
#####################################################################################################################################

import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
def getTrainValidDatagens(train_dir , valid_dir):
  train_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range = 40,
      width_shift_range = 0.2,
      height_shift_range = 0.2,
      shear_range = 0.2,
      zoom_range = 0.2,
      horizontal_flip = True,
      fill_mode = 'nearest')
  
  train_images = train_datagen.flow_from_directory(
      train_dir,
      target_size = (150, 150)
      #class_mode = 'categorical'
  )

  valid_datagen = ImageDataGenerator(rescale = 1./255)

  valid_images = valid_datagen.flow_from_directory(
      valid_dir,
      target_size = (150, 150)
      #class_mode = 'categorical'
  )

  return train_images , valid_images

#Create train and test datasets from downloaded zip file
The contents of the .zip are extracted to the base directory /tmp/, which contains train (/tmp/rps) and validation subdirectories (/tmp/rps-test-set) for the training and validation datasets (see the Machine Learning Crash Course for a refresher on training, validation, and test sets), which in turn each contain rock, paper and scissor subdirectories.

In short: The training set is the data that is used to tell the neural network model that 'this is what a rock looks like', 'this is what a paper looks like' etc. The validation data set is images of rock, paper and scissors that the neural network will not see as part of the training, so you can test how well or how badly it does in evaluating if an image contains a cat or a dog.

One thing to pay attention to in this sample: We do not explicitly label the images of rocks, paper or scissors. If you remember with the handwriting example earlier, we had labelled 'this is a 1', 'this is a 7' etc. Later you'll see something called an ImageGenerator being used -- and this is coded to read images from subdirectories, and automatically label them from the name of that subdirectory. So, for example, you will have a 'training' directory containing a 'rock', 'paper' and 'scissors' directory. ImageGenerator will label the images appropriately for you, reducing a coding step.

Let's define each of these directories:

In [0]:
####################################################################################################
#Download signs dataset and extract train, test data from the appropraite directories              
####################################################################################################

import os
import zipfile

local_zip = '/tmp/rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

local_zip = '/tmp/rps-test-set.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

rock_dir = os.path.join('/tmp/rps/rock')
paper_dir = os.path.join('/tmp/rps/paper')
scissors_dir = os.path.join('/tmp/rps/scissors')

#Peak in the dataset
print('total training rock images:', len(os.listdir(rock_dir)))
print('total training paper images:', len(os.listdir(paper_dir)))
print('total training scissors images:', len(os.listdir(scissors_dir)))

rock_files = os.listdir(rock_dir)
print(rock_files[:10])

paper_files = os.listdir(paper_dir)
print(paper_files[:10])

scissors_files = os.listdir(scissors_dir)
print(scissors_files[:10])

%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pic_index = 2

next_rock = [os.path.join(rock_dir, fname) 
                for fname in rock_files[pic_index-2:pic_index]]
next_paper = [os.path.join(paper_dir, fname) 
                for fname in paper_files[pic_index-2:pic_index]]
next_scissors = [os.path.join(scissors_dir, fname) 
                for fname in scissors_files[pic_index-2:pic_index]]

for i, img_path in enumerate(next_rock+next_paper+next_scissors):
  #print(img_path)
  img = mpimg.imread(img_path)
  plt.imshow(img)
  plt.axis('Off')
  plt.show()

#Train and validation generator for fitting the model
#train data is stored in '/tpm/rps' and validation data in '/tms/rps-test-set'
rps_train_generator_images , rps_valid_generator_images = getTrainValidDatagens(train_dir = '/tmp/rps', valid_dir = '/tmp/rps-test-set')

In [0]:
####################################################################################################
#Download Cats-v-Dogs dataset from Kaggle and create tran and test datasets from the raw data file 
####################################################################################################

import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

# If the URL doesn't work, visit https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765
# And right click on the 'Download Manually' link to get a new URL to the dataset

# Note: This is a very large dataset and will take time to download

!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()
print(len(os.listdir('/tmp/PetImages/Cat/')))
print(len(os.listdir('/tmp/PetImages/Dog/')))


In [0]:
try:
    os.mkdir('/tmp/cats-v-dogs')
    os.mkdir('/tmp/cats-v-dogs/training')
    os.mkdir('/tmp/cats-v-dogs/testing')
    os.mkdir('/tmp/cats-v-dogs/training/cats')
    os.mkdir('/tmp/cats-v-dogs/training/dogs')
    os.mkdir('/tmp/cats-v-dogs/testing/cats')
    os.mkdir('/tmp/cats-v-dogs/testing/dogs')
except OSError:
    pass

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[:testing_length]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)

CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"

DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

TRAIN_DIR = "/tmp/cats-v-dogs/training/"
VALID_DIR = "/tmp/cats-v-dogs/testing/"

#Train and validation generator for fitting the model
#train data is stored in '/tmp/cats-v-dog/training' and validation data in '/tmp/cats-d-dogs/testing'
catsDogs_train_generator_images , catsDogs_valid_generator_images = getTrainValidDatagens(train_dir = TRAIN_DIR, valid_dir = VALID_DIR)

The data for this exercise is available at: https://www.kaggle.com/datamunge/sign-language-mnist/home

Sign up and download to find 2 CSV files: sign_mnist_test.csv and sign_mnist_train.csv -- You will upload both of them using this button before you can continue.

In [0]:
##################################################################################
# Upload the raw datafile from computer's HD and generate train and test datasets 
##################################################################################
from google.colab import files

upoaded = files.upload()

In [0]:
def get_data(filename):
  with open(filename) as training_file:
    csv_reader = csv.reader(training_file, delimiter= ',')
    first_line = True
    temp_images = []
    temp_labels = []
    for row in csv_reader:
      if first_line:
        # print("Ignoring first line")
        first_line = False
      else:
        temp_labels.append(row[0])
        image_data = row[1:785]
        image_data_as_array = np.array_split(image_data, 28)
        temp_images.append(image_data_as_array)
    images = np.array(temp_images).astype('float')
    labels = np.array(temp_labels).astype('float')
  return images, labels

training_images, training_labels = get_data('sign_mnist_train.csv')
testing_images, testing_labels = get_data('sign_mnist_test.csv')

print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

training_images = np.exapnd_dims(training_images, axis=3)
testing_images = np/expand_dims(testing_images, axis=3)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(
    rescale=1. / 255)

##Train and test image datasets to be fed in the model
uploaded_train_generated_images = train_datagen.flow(training_images, training_labels, batch_size=32)
uploaded_test_generated_images = validation_datagen.flow(testing_images, testing_labels, batch_size=32)


In [0]:
######################################################
# Functions for call back and plotting model results 
######################################################

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

def plotCharts(history):
  acc = history.history['accuracy']
  loss = history.history['loss']
  val_acc = history.history['val_accuracy']
  val_loss = history.history['val_loss']
  epochs = range(len(acc))

  plt.plot(epochs, acc, 'r', label='Training accuracy')
  plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
  plt.title('Training and validation accuracy')
  plt.figure()

  plt.plot(epochs, loss, 'r', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.figure()

# Define a Callback class that stops training once accuracy reaches 99.9%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.999):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True


In [0]:
###################################################
#Binary, Categorical and transfer learning models 
###################################################

def runThreeClassModel(train_generator_images , valid_generator_images, epochs):
  model = tf.keras.models.Sequential([
      # Note the input shape is the desired size of the image 150x150 with 3 bytes color
      # This is the first convolution
      tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      # The second convolution
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # The third convolution
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # The fourth convolution
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # Flatten the results to feed into a DNN
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dropout(0.5),
      # 512 neuron hidden layer
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(3, activation='softmax')
  ])

  model.summary()
  model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
  callbacks = myCallback()
  history = model.fit(train_generator_images, epochs = epochs, validation_data = valid_generator_images, verbose = 1, callbacks=[callbacks])
  plotCharts(history)
  #visualize(model)

def runBinaryClassModel(train_generator_images , valid_generator_images, epochs):
  model = tf.keras.models.Sequential([
      # Note the input shape is the desired size of the image 150x150 with 3 bytes color
      # This is the first convolution
      tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      # The second convolution
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # The third convolution
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # The fourth convolution
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # Flatten the results to feed into a DNN
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dropout(0.5),
      # 512 neuron hidden layer
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])

  model.summary()
  model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
  callbacks = myCallback()
  history = model.fit(train_generator_images, epochs = epochs, validation_data = valid_generator_images, verbose = 1, callbacks=[callbacks])
  plotCharts(history)
  #visualize(model)

#Model for transfer learning using InceptionV3
def runTransferInception(train_generator_images , valid_generator_images, epochs):
  # Download the inception v3 weights
  !wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

  # Import the inception model  
  from tensorflow.keras.applications.inception_v3 import InceptionV3
  from tensorflow.keras import Model
  # Create an instance of the inception model from the local pre-trained weights
  local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

  pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                                 include_top = False,
                                 weights = None)
  
  pre_trained_model.load_weights(local_weights_file)

  #Make all layers in pre-trained model non-trainable
  from tensorflow.keras import layers
  for layer in pre_trained_model.layers:
    layer.trainable = False
  
  #pre_trained_model.summary()

  last_layer = pre_trained_model.get_layer('mixed7')
  print('last layer output shape: ', last_layer.output_shape)
  last_output = last_layer.output
  
  from tensorflow.keras.optimizers import RMSprop
 # Flatten the output layer to 1 dimension
  x = layers.Flatten()(last_output)
  # Add a fully connected layer with 1,024 hidden units and ReLU activation
  x = layers.Dense(1024, activation='relu')(x)
  # Add a dropout rate of 0.2
  x = layers.Dropout(0.2)(x)                  
  # Add a final sigmoid layer for classification
  x = layers.Dense  (3, activation='sigmoid')(x)           

  model = Model( pre_trained_model.input, x) 

  model.compile(optimizer = RMSprop(lr=0.0001), 
                loss = 'categorical_crossentropy', 
                metrics = ['accuracy'])

  model.summary()

  callbacks=myCallback()
  history = model.fit_generator(
      train_generator_images,
      validation_data = valid_generator_images,
      steps_per_epoch = 100,
      epochs=epochs,
      validation_steps= 50,
      verbose = 2,
      callbacks=[callbacks],
  )
  plotCharts(history)

In [0]:
#Run models on rps data

runThreeClassModel(rps_train_generator_images, rps_valid_generator_images, epochs=1)
runTransferInception(rps_train_generator_images, rps_valid_generator_images, epochs=1)

#Run models on cats and dogs data 

runBinaryClassModel(catsDogs_train_generator_images, catsDogs_valid_generator_images, epochs=1)
runTransferInception(catsDogs_train_generator_images, catsDogs_valid_generator_images, epochs=1)

#Visualizing Intermediate Representations
To get a feel for what kind of features our convnet has learned, one fun thing to do is to visualize how an input gets transformed as it goes through the convnet.

Let's pick a random cat or dog image from the training set, and then generate a figure where each row is the output of a layer, and each image in the row is a specific filter in that output feature map. Rerun this cell to generate intermediate representations for a variety of training images.

In [0]:
#Function for visualizing intermediate images of convolutions

def visualize(model):
  #visualize responses
  import numpy as np
  import random
  from   tensorflow.keras.preprocessing.image import img_to_array, load_img

  # Let's define a new Model that will take an image as input, and will output
  # intermediate representations for all layers in the previous model after
  # the first.
  successive_outputs = [layer.output for layer in model.layers[1:]]

  #visualization_model = Model(img_input, successive_outputs)
  visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

  # Let's prepare a random input image of a cat or dog from the training set.
  cat_img_files = [os.path.join(train_cats_dir, f) for f in train_cat_fnames]
  dog_img_files = [os.path.join(train_dogs_dir, f) for f in train_dog_fnames]

  img_path = random.choice(cat_img_files + dog_img_files)
  img = load_img(img_path, target_size=(150, 150))  # this is a PIL image

  x   = img_to_array(img)                           # Numpy array with shape (150, 150, 3)
  x   = x.reshape((1,) + x.shape)                   # Numpy array with shape (1, 150, 150, 3)

  # Rescale by 1/255
  x /= 255.0

  # Let's run our image through our network, thus obtaining all
  # intermediate representations for this image.
  successive_feature_maps = visualization_model.predict(x)

  # These are the names of the layers, so can have them as part of our plot
  layer_names = [layer.name for layer in model.layers]

  # -----------------------------------------------------------------------
  # Now let's display our representations
  # -----------------------------------------------------------------------
  for layer_name, feature_map in zip(layer_names, successive_feature_maps):
    
    if len(feature_map.shape) == 4:
      
      #-------------------------------------------
      # Just do this for the conv / maxpool layers, not the fully-connected layers
      #-------------------------------------------
      n_features = feature_map.shape[-1]  # number of features in the feature map
      size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)
      
      # We will tile our images in this matrix
      display_grid = np.zeros((size, size * n_features))
      
      #-------------------------------------------------
      # Postprocess the feature to be visually palatable
      #-------------------------------------------------
      for i in range(n_features):
        x  = feature_map[0, :, :, i]
        x -= x.mean()
        x /= x.std ()
        x *=  64
        x += 128
        x  = np.clip(x, 0, 255).astype('uint8')
        display_grid[:, i * size : (i + 1) * size] = x # Tile each filter into a horizontal grid

      #-----------------
      # Display the grid
      #-----------------

      scale = 20. / n_features
      plt.figure( figsize=(scale * n_features, scale) )
      plt.title ( layer_name )
      plt.grid  ( False )
      plt.imshow( display_grid, aspect='auto', cmap='viridis' ) 

As you can see we go from the raw pixels of the images to increasingly abstract and compact representations. The representations downstream start highlighting what the network pays attention to, and they show fewer and fewer features being "activated"; most are set to zero. This is called "sparsity." Representation sparsity is a key feature of deep learning.

These representations carry increasingly less information about the original pixels of the image, but increasingly refined information about the class of the image. You can think of a convnet (or a deep network in general) as an information distillation pipeline.

In [0]:
#Upload rock/paper/scissor images and run models on them to predict it

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)
